## Generate data from sims.

In [ ]:
from shared.preface import *
import shared.functions as fct

# Initialize parameters and files.
PRE = PRE(
    # sim='L006N188', 
    sim='L012N376', 
    z0_snap=36, z4_snap=13, DM_lim=1000,
    sim_dir=SIM_ROOT, sim_ver=SIM_TYPE,
    phis=10, thetas=10, vels=100,
    pre_CPUs=6, sim_CPUs=6
)

mass_gauge = 12.3
mass_range = 0.3
size = 1
hname = f'1e+{mass_gauge}_pm{mass_range}Msun'
fct.halo_batch_indices(
    PRE.Z0_STR, mass_gauge, mass_range, 'halos', size, 
    hname, PRE.SIM_DIR, PRE.OUT_DIR
)
halo_batch_IDs = np.load(f'{PRE.OUT_DIR}/halo_batch_{hname}_indices.npy')
halo_batch_params = np.load(f'{PRE.OUT_DIR}/halo_batch_{hname}_params.npy')
halo_num = len(halo_batch_params)

print('********Number density band********')
print('Halo batch params (Rvir,Mvir,cNFW):')
print(halo_batch_params)
print('***********************************')


mass_ticks = 100
mass_range_eV = np.geomspace(0.01, 0.3, mass_ticks)*eV  # 10 to 300 meV

etas_arr = []
for halo_j in range(halo_num):

    # Load velocities.
    tot_name = f'{hname}_halo{halo_j}'
    fname = f'{PRE.NUS}nus_{tot_name}'
    u_all = fct.load_sim_data(PRE.OUT_DIR, fname, 'velocities')

    # Calculate number density.
    out_file = f'{PRE.OUT_DIR}/number_densities_{PRE.NUS}nus_{tot_name}.npy'
    fct.number_densities_mass_range(
        u_all, mass_range_eV, out_file,
        average=False, m_start=0.01, z_start=0.5
    )

    etas = np.load(f'{out_file}')/N0
    etas_arr.append(etas)

etas_sim = np.array(etas_arr).reshape(halo_num, mass_ticks)


# Load data for MW halo only of smooth simulation
nus_smooth = 10000
MW_HALO = True
VC_HALO = False
AG_HALO = False
halos = 'MW'*MW_HALO + '+VC'*VC_HALO + '+AG'*AG_HALO
sim = 'LinfNinf'
smooth_name = f'nus_{nus_smooth}_halos_{halos}_{SOLVER}'
u_all_MW = fct.load_sim_data(sim, smooth_name, 'velocities')

output = f'{sim}/number_densities_{smooth_name}.npy'
fct.number_densities_mass_range(
    u_all_MW, mass_range_eV, output, 
    average=False, m_start=0.01, z_start=0.5
)
etas_smooth = np.load(f'{output}')/N0

## Plotting.

In [ ]:
fig_dir = f'{os.getcwd()}/figures'
fname = f'{SIM_TYPE}_{hname}'
fct.plot_eta_band(etas_sim, etas_smooth, mass_range_eV, fig_dir, fname, show=True)

In [ ]:
fig_dir = f'{os.getcwd()}/figures'
fname = f'{SIM_TYPE}_{hname}'
fct.plot_eta_z_back_1Halo(u_all, NU_MASSES, fig_dir, fname, show=True)


In [ ]:
fig_dir = f'{os.getcwd()}/figures'
fname = f'{SIM_TYPE}_{hname}'
fct.plot_phase_space_1Halo(
    u_all, NU_MASSES, halo_batch_params[0], 
    PRE.Vs, PRE.PHIs, PRE.THETAs, PRE.LOWER, PRE.UPPER,
    fig_dir, fname, show=True
)

In [ ]:
fig_dir = f'{os.getcwd()}/figures'
fname = f'{SIM_TYPE}_{hname}'
fct.plot_number_density_integral(
    u_all, NU_MASSES, 
    PRE.Vs, PRE.PHIs, PRE.THETAs, PRE.LOWER, PRE.UPPER,
    fig_dir, fname, show=True
)